In [1]:
from elasticsearch import Elasticsearch, exceptions
import json, time

In [2]:
DOMAIN = "localhost"
PORT = 9200

In [3]:
host = str(DOMAIN) + ":" + str(PORT)

In [4]:
client = Elasticsearch(host)

In [5]:
try:
    info = json.dumps(client.info(), indent=4)
    print ("Elasticsearch client info():", info)

except exceptions.ConnectionError as err:
    print ("\nElasticsearch info() ERROR:", err)
    print ("\nThe client host:", host, "is invalid or cluster is not running")
    client = None

Elasticsearch client info(): {
    "version": {
        "build_hash": "22e1767283e61a198cb4db791ea66e3f11ab9910",
        "build_flavor": "default",
        "build_type": "tar",
        "minimum_index_compatibility_version": "6.0.0-beta1",
        "build_date": "2019-09-27T08:36:48.569419Z",
        "build_snapshot": false,
        "number": "7.4.0",
        "minimum_wire_compatibility_version": "6.8.0",
        "lucene_version": "8.2.0"
    },
    "tagline": "You Know, for Search",
    "cluster_uuid": "U3E6ysB6SHWDQs0RFHccDA",
    "name": "elk",
    "cluster_name": "elasticsearch"
}


In [6]:
if client == None:
    exit()

In [7]:
print(client.indices.get_alias("*"))

{'twitter-corona-': {'aliases': {}}, '.kibana_1': {'aliases': {'.kibana': {}}}, 'twitter-yuvi': {'aliases': {}}, 'twitter': {'aliases': {}}, '.kibana_task_manager_1': {'aliases': {'.kibana_task_manager': {}}}, 'twitter-corona': {'aliases': {}}, '.apm-agent-configuration': {'aliases': {}}, 'twitter-corona-3.': {'aliases': {}}, 'twitter-corona-4.': {'aliases': {}}}


In [8]:
import json

In [10]:
# indices = ['twitter-corona-3.','twitter-corona','twitter-corona-']
# sorry for the terribly named indices 
# data collection was started in march with twitter-corona, then twitter-corona- and then the last one
indices = ['twitter-corona-']
doc_count = 0
filecount = 1
filename = ""
PATH = "./"
f = None
# iterate over the list of Elasticsearch indices
for num, index in enumerate(indices):
    # declare a filter query dict object    
    # make a search() request to get all docs in the index
    match_all = {
        "size": 1000,
        "query": {
            "match_all": {}
        }
    }
    resp = client.search(
        index = index,
        body = match_all,
        scroll = '3s' # length of time to keep search context
    )
    old_scroll_id = resp['_scroll_id']
    while len(resp['hits']['hits']):
        resp = client.scroll(
            scroll_id = old_scroll_id,
            scroll = '2s' # length of time to keep search context
        )
        if old_scroll_id != resp['_scroll_id']:
            print ("NEW SCROLL ID:", resp['_scroll_id'])

        # keep track of pass scroll _id
        old_scroll_id = resp['_scroll_id']

        if len(resp['hits']['hits']) > 0:
            prev_filename = filename
            filename = str(resp['hits']['hits'][0]['_source']['tweet_created_at'][:10]) + ".txt"
#             print(filename)
            if filename != prev_filename:
                if f!= None:
                    f.close()
                f = open(PATH + filename, "a")
        for doc in resp['hits']['hits']:
            print ("\n", doc['_id'], doc['_source'])
#             data.append(doc['_id'])
#             f.write(json.dumps(doc)+"\n")  
#             f.write(doc['_id']+"\n")
#             f.write("hey \n")
#             print ("\n", doc['_id'], doc['_source']['tweet_created_at'])
            doc_count += 1
#         print ("DOC COUNT:", doc_count)
        if (doc_count % 10000) == 0:
            print ("DOC COUNT:", doc_count)  


 1250018378116476930 {'author': {'author_location': 'Ado Ekiti, Nigeria', 'author_friends_count': 685, 'author_pseudo': 'teaborehe', 'author_name': 'Temiloluwa Oyebola', 'author_followers_count': 498, 'author_account_creation': '2012-05-12T13:04:47.000Z', 'author_description': 'medical laboratory scientist. live and let live. Manchester United will not be the death of me.', 'author_id': '578119671'}, 'quoted_status_permalink': {'expanded': 'https://twitter.com/legitngnews/status/1249991724505669632', 'display': 'twitter.com/legitngnews/st…', 'url': 'https://t.co/UIYZklYI57'}, 'favorite_count': 0, 'tweet_created_at': '2020-04-14T11:09:40.000Z', 'retweet_count': 0, 'tweet_id': '1250018378116476930', 'inserted_in_es_at': '2020-04-14T11:09:40.000Z', 'tweet_content': 'LMAO,  what is this?  What will the face mask achieve?  \U0001f923\U0001f923\U0001f923\U0001f923\U0001f923\U0001f923 this must be a joke because kini nonsense yi', 'tweet_lang': 'en', 'is_a_retweet': 'false'}

 12500183786698

In [53]:
#     match_all = {
#             "size": 1000,
#         "sort": ['tweet_created_at'],
#         "query": {
#             "match_all": {}
#         }
#     }


2020-03-14-1.txt  demofile2.txt  getdocs.ipynb	ids


In [ ]:
    match_all = {
#         "_source": ["tweet_id","tweet_created_at"],
        "size": 1000,
        "sort": ['tweet_created_at'],
#         "query": {
#             "match_phrase": {"author.author_location": "India"}
#         }
        
    }